# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,4.97,45,20,2.06,US,1682471206
1,1,ixtapa,20.7000,-105.2000,25.13,78,75,3.60,MX,1682471206
2,2,bethel,41.3712,-73.4140,10.92,65,0,2.06,US,1682471024
3,3,hawaiian paradise park,19.5933,-154.9731,27.73,69,0,5.66,US,1682471206
4,4,nogliki,51.8333,143.1667,2.47,86,100,5.97,RU,1682471207


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    frame_width = 500,
    frame_height = 500,
    tiles = 'OSM',
    color = 'City',
    size = 'Humidity'
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
best_cities = city_data_df[(city_data_df['Max Temp']<27) & (city_data_df['Max Temp']>21) & 
             (city_data_df['Wind Speed']<4.5) &
             (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
best_cities.dropna()
# Display sample data
best_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,khasab,26.1799,56.2477,25.89,59,0,2.06,OM,1682471236
173,173,prado,-17.3411,-39.2208,23.42,90,0,1.08,BR,1682471109
261,261,ouargla,31.9493,5.3250,25.83,20,0,4.44,DZ,1682471278
311,311,crane,31.3974,-102.3501,26.47,17,0,3.37,US,1682471294
463,463,sale,34.0389,-6.8166,22.58,82,0,0.00,MA,1682471337


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities[['City','Country','Lat','Lng','Humidity']].copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
113,khasab,OM,26.1799,56.2477,59,
173,prado,BR,-17.3411,-39.2208,90,
261,ouargla,DZ,31.9493,5.3250,20,
311,crane,US,31.3974,-102.3501,17,
463,sale,MA,34.0389,-6.8166,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey':geoapify_key,
    'limit':20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty and make it a JSON response
    name_address = requests.get(base_url,params=params).json()
   
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
khasab - nearest hotel: "Extra Divers Villa"
prado - nearest hotel: Pousada Recanto do Prado
ouargla - nearest hotel: LYNATEL
crane - nearest hotel: No hotel found
sale - nearest hotel: Le dawliz
ain sefra - nearest hotel: فندق مكثر
el abiodh sidi cheikh - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
113,khasab,OM,26.1799,56.2477,59,"""Extra Divers Villa"""
173,prado,BR,-17.3411,-39.2208,90,Pousada Recanto do Prado
261,ouargla,DZ,31.9493,5.3250,20,LYNATEL
311,crane,US,31.3974,-102.3501,17,No hotel found
463,sale,MA,34.0389,-6.8166,82,Le dawliz


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    frame_width = 500,
    frame_height = 500,
    tiles = 'OSM',
    color = 'City',
    size = 'Humidity',
    hover_cols = ['Hotel Name','Country']
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)